In [2]:
import os, sys
import time
import argparse
import multiprocessing
import numpy as np

from utils.io import readcoo2mat
from all_parser import *


import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from scipy.sparse import coo_matrix

In [3]:
torch.__version__

'1.13.1'

In [40]:

def read_data(data_file, norm_file, out_dir, resolution):
    filename = os.path.basename(data_file).split('.')[0] + '.npz'
    out_file = os.path.join(out_dir, filename)
    try:
        HiC, idx = readcoo2mat(data_file, norm_file, resolution)
    except:
        print(f'Abnormal finle: {norm_file}')
    np.savez_compressed(out_file, hic=HiC, compact=idx)
    print('Saving file:', out_file)



In [41]:
data_file = "data/RaoHiC/raw/GM12878/10kb_resolution_intrachromosomal/chr18/MAPQGE30/chr18_10kb.RAWobserved" 
norm_file = "data/RaoHiC/raw/GM12878/10kb_resolution_intrachromosomal/chr18/MAPQGE30/chr18_10kb.KRnorm" 
out_dir = "data/RaoHiC/mat/GM12878" 
resolution = 10000

read_data(data_file, norm_file, out_dir, resolution)


data/RaoHiC/raw/GM12878/10kb_resolution_intrachromosomal/chr18/MAPQGE30/chr18_10kb.RAWobserved data/RaoHiC/raw/GM12878/10kb_resolution_intrachromosomal/chr18/MAPQGE30/chr18_10kb.KRnorm 10000


NameError: name 'HiC' is not defined

In [ ]:
#python data_aread.py -c GM12878

In [4]:

cell_line = "GM12878"
resolution = "10kb"
map_quality = "MAPQGE30"
#后缀
postfix = ["KRnorm", 'RAWobserved']

In [5]:
pool_num = 23 if multiprocessing.cpu_count() > 23 else multiprocessing.cpu_count()

#原始数据的路径
raw_dir = os.path.join(root_dir, 'raw', cell_line)

In [19]:
norm_files = []
data_files = []
for root, dirs, files in os.walk(raw_dir):
    if len(files) > 0:
        if (resolution in root) and (map_quality in root):
            for f in files:
                if (f.endswith(postfix[0])):
                    norm_files.append(os.path.join(root, f))
                elif (f.endswith(postfix[1])):
                    data_files.append(os.path.join(root, f))

out_dir = os.path.join(root_dir, 'mat', cell_line)

mkdir(out_dir)
print(f'Start reading data, there are {len(norm_files)} files ({resolution}).')
print(f'Output directory: {out_dir}')


start = time.time()
pool = multiprocessing.Pool(processes=pool_num)
print(f'Start a multiprocess pool with process_num={pool_num} for reading raw data')
#多线程
for data_fn, norm_fn in zip(data_files, norm_files):
    pool.apply_async(read_data, (data_fn, norm_fn, out_dir, res_map[resolution]))
pool.close()
pool.join()
print(f'All reading processes done. Running cost is {(time.time()-start)/60:.1f} min.')

Start reading data, there are 23 files (10kb).
Output directory: data/RaoHiC/mat/GM12878
